# Chapter 13. Interfaces, Protocols, and ABCs

"Program to an interface, not an implementation."

OOP is all about interfaces.

Ở Python 3.8, có 4 cách để định nghĩa interfaces, được mô tả trong Typing Map

- Duck typing (Chap 1)
    - Default
    - Tư tưởng: "If it walks like a duck and quacks like a duck, it must be a duck".
        -  Kiểu hoặc lớp (class) của một đối tượng (object) ít quan trọng hơn phương thức mà nó định nghĩa.
- Goose typing (Chap 13)
    - Supported by abstract base classes (ABCs)
    - Dựa trên runtime check ABCs của object
- Static typing (chap 8, 15)
    - Hướng tiếp cận của statical languages (Java, C)
    - Hỗ trợ bởi `typing` module
    - Enforced by external type checkers
- Static duck typing
    - Phổ biến ở Go
    - Hỗ trợ bởi `typing.Protocol`
    - Enforced by external type checkers

![image.png](image/13-1.png)

Chương này nói về duck typing, goose typing, and static duck typing
- static typing chỉ sử dụng các types cụ thể thay vì interface abstractions

## Two Kinds of Protocols

So sánh 2 loại Typing (bên trái) với Protocol

- Protocols
    - VD: Trong network protocols là `GET`, `PUT`
    - VD: Trong sequence protocol - method cho phép object như 1 seq
        - `__getitem__` method
        - `__len__` method (optional)
    - --> Protocol is an “informal interface.”

- Dynamic protocol
    - informal protocols
    - implicit (ngầm định)
- Static protocal
    - từ Python 3.8 (static duck tying)
    - explicit (rõ ràng): `typing.Protocol` subclass, ABCs

2 điểm khác biệt chính giữa 2 loại:
- Implement
    - Dynamic protocal: Object có thể chỉ cần impl 1 phần mà vẫn useful
    - Static protocol: Object phải impl mọi method trong protocal class
- Verify
    - Dynamic protocal: NaN
    - Static protocol: By static type checkers

The rest of this chapter covers dynamic and static protocols, as well as ABCs.

## Programming Ducks

Deep driver
Make it safer while preserving its major strength (flexibility)

Bắt đầu dynamic protocols với __sequence__ và __iterable__ protocols.

### Python Digs(đào) Sequences

Triết lý của Python Data Model là work với nhiều dynamic protocols thiết yếu nhất có thể.

Hình 13-2 cho thấy cách `Sequence` interface được chính thức hóa dưới dạng ABC. Mô tả những gì mà một full-fledged `Sequence` sẽ hỗ trợ.

![image.png](image/13-2.png)

Từ 13-2, thấy rằng subclass đúng của Seg phải imple `__getitem__` và `__len__`. Những methods khác là cụ thể -> subclass sẽ kế thừa nó/viết lại 1 cái phù hợp hơn.
- VD: Khi ko có `__iter__`, Python dự phòng rằng nếu có `__getitem__`, nó sẽ chạy index start ở 0.

TIP

The ABCs are useful as starting points for new classes, and to support explicit type checking at runtime (a.k.a. goose typing) as well as type hints for static type checkers.

Ví dụ sau không là subclass abc.Sequence, nhưng imple `__getitem__` và `__len__`

In [1]:
from collections import namedtuple

Card = namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                       for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

### Monkey Patching: Implementing a Protocol at Runtime

Monkey patching is dynamically changing a module, class, or function at runtime, to add features or fix bugs.

`FrenchDeck` ở trên thiếu đi tính năng `shuffle`. Nhưng theo Pythonic, vì ta đã làm nó act như 1 Seq -> nó ko cần viết 1 hàm shuffle của nó, mà có thể dùng `random.shuffle`

TIP

Khi bạn tuân theo các protocols đã được thiết lập, bạn sẽ cải thiện cơ hội tận dụng thư viện tiêu chuẩn và mã của bên thứ ba,, thanks to duck typing.

In [5]:
# shuffle operates in place, by swapping items inside the collection
from random import shuffle
deck = FrenchDeck()
list(deck)

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades'),
 Card(rank='5', suit='spades'),
 Card(rank='6', suit='spades'),
 Card(rank='7', suit='spades'),
 Card(rank='8', suit='spades'),
 Card(rank='9', suit='spades'),
 Card(rank='10', suit='spades'),
 Card(rank='J', suit='spades'),
 Card(rank='Q', suit='spades'),
 Card(rank='K', suit='spades'),
 Card(rank='A', suit='spades'),
 Card(rank='2', suit='diamonds'),
 Card(rank='3', suit='diamonds'),
 Card(rank='4', suit='diamonds'),
 Card(rank='5', suit='diamonds'),
 Card(rank='6', suit='diamonds'),
 Card(rank='7', suit='diamonds'),
 Card(rank='8', suit='diamonds'),
 Card(rank='9', suit='diamonds'),
 Card(rank='10', suit='diamonds'),
 Card(rank='J', suit='diamonds'),
 Card(rank='Q', suit='diamonds'),
 Card(rank='K', suit='diamonds'),
 Card(rank='A', suit='diamonds'),
 Card(rank='2', suit='clubs'),
 Card(rank='3', suit='clubs'),
 Card(rank='4', suit='clubs'),
 Card(rank='5', suit='clubs'),
 Card(rank='6', 

In [6]:
shuffle(deck)
# error vì FrenchDeck chỉ imple immutable seq protocal --> không thể shuffle in place
# --> Muốn nó là mutable seq, cần cung cấp __setitem__

TypeError: 'FrenchDeck' object does not support item assignment

In [8]:
# Fix nó ở runtime, tương tác với console
# deck, position, card thay vì self, key, value để chỉ rằng methods nên bắt đầu từ 1 func đơn giản và đặt tên tham số đầu là self chỉ là 1 quy ước --> đặt vậy OK vì mình viết nó ở console.
def set_card(deck, position, card):
    deck._cards[position] = card

FrenchDeck.__setitem__ = set_card # <-- Monkey Patching
shuffle(deck)
deck[:5]

[Card(rank='K', suit='spades'),
 Card(rank='J', suit='diamonds'),
 Card(rank='Q', suit='clubs'),
 Card(rank='9', suit='hearts'),
 Card(rank='9', suit='diamonds')]

Phần tiếp theo show một số code patterns để phát hiện các dynamic protocols mà không cần dùng đến các kiểm tra rõ ràng.

### Defensive Programming and “Fail Fast”

In a dynamically typed language, “fail fast” is excellent advice for safer and easier-to-maintain programs.
- Failing fast means raising runtime errors as soon as possible
    - VD: rejecting invalid arguments right a the beginning of a function body.



In [ ]:
# VD
# Ngay lập tức build list từ iterable
# Nếu iterable sai kiểu , ngay lập tức fail fast: TypeError
def __init__(self, iterable):
    self._balls = list(iterable)

# Nếu không làm vậy, chương trình sẽ vẫn raise khi 1 hàm nào đó gọi self._balls mà không phải là list --> khi đó khó tìm ra root cause.

# Tuy nhiên, gọi list() có thể tệ nếu ta cần change nó in-place như ví dụ shuffle.
# Khi đó, runtime check `isinstance(x, abc.MutableSequence)` hoặc `len()` sẽ cần đến

Ở ví dụ trên, type hint sẽ catch đc problem sớm hơn, nhưng không phải catch đc mọi thứ.
- `Any` type
- Không được enforced ở runtime


--> Fail fast là tuyến phòng thủ cuối

In [ ]:
# Ví dụ 2: Defensive code + duck typing để handle việc thay đổi biến field_names

# Example 13-5. Duck typing to handle a string or an iterable of strings
field_names = 'huy, mac'
try:
        field_names = field_names.replace(',', ' ').split()
except AttributeError:
    pass
field_names = tuple(field_names)
if not all(s.isidentifier() for s in field_names):
    raise ValueError('field_names must all be valid identifiers')

--> duck typing is more expressive than static type hints
```python
def namedtuple(
        typename: str,
        field_names: Union[str, Iterable[str]],
        *,
        # rest of signature omitted
```

Giờ, move sang loại runtime type check rõ ràng hơn: Goose typing

## Goose Typing

ABCs for runtime type check

Python doesn’t have an `interface` keyword. We use `abstract base classes (ABCs`) to define interfaces for explicit type checking at runtime—also supported by static type checkers.
- ABCs introduce virtual subclasses
    - which are classes that don’t inherit from a class but are still recognized by `isinstance()` and `issubclass()`

goose typing đòi hỏi
- Subclassing từ ABCs để làm nó rõ ràng rằng bạn đang triển khai một giao diện được xác định trước đó
- Runtime type check sử dụng ABCs thay vì các lớp cụ thể làm đối số thứ hai cho hàm `isinstance` và `issubclass`.
    - Nếu dùng lớp cụ thể -> type check giời hạn tính đa hình trong OOP

ABCs có nghĩa là gói gọn các khái niệm, được giới thiệu qua 1 framework. Hầu hết ko cần viết thêm ABCs mới, chỉ cần sử dụng các ABCs đã có sẵn

### Subclassing an ABC


In [3]:
# Example 13-6. frenchdeck2.py: FrenchDeck2, a subclass of collections.MutableSequence

from collections import namedtuple, abc

Card = namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                       for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):  # <1> Đủ để ta shuffle rồi
        self._cards[position] = value

from random import shuffle
deck = FrenchDeck2()
list(deck)

TypeError: Can't instantiate abstract class FrenchDeck2 with abstract methods __delitem__, insert

In [4]:
# Example 13-6. frenchdeck2.py: FrenchDeck2, a subclass of collections.MutableSequence

from collections import namedtuple, abc

Card = namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                       for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):  # <1> Đủ để ta shuffle rồi
        self._cards[position] = value

    def __delitem__(self, position):  # <2> Nhưng subclassing abc.MutableSequence đòi hỏi ta cần imple del nữa (khác với Duck typing)
        del self._cards[position]

    def insert(self, position, value):  # <3> Và yêu cầu cần imple cả insert
        self._cards.insert(position, value)
from random import shuffle
deck = FrenchDeck2()
list(deck)

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades'),
 Card(rank='5', suit='spades'),
 Card(rank='6', suit='spades'),
 Card(rank='7', suit='spades'),
 Card(rank='8', suit='spades'),
 Card(rank='9', suit='spades'),
 Card(rank='10', suit='spades'),
 Card(rank='J', suit='spades'),
 Card(rank='Q', suit='spades'),
 Card(rank='K', suit='spades'),
 Card(rank='A', suit='spades'),
 Card(rank='2', suit='diamonds'),
 Card(rank='3', suit='diamonds'),
 Card(rank='4', suit='diamonds'),
 Card(rank='5', suit='diamonds'),
 Card(rank='6', suit='diamonds'),
 Card(rank='7', suit='diamonds'),
 Card(rank='8', suit='diamonds'),
 Card(rank='9', suit='diamonds'),
 Card(rank='10', suit='diamonds'),
 Card(rank='J', suit='diamonds'),
 Card(rank='Q', suit='diamonds'),
 Card(rank='K', suit='diamonds'),
 Card(rank='A', suit='diamonds'),
 Card(rank='2', suit='clubs'),
 Card(rank='3', suit='clubs'),
 Card(rank='4', suit='clubs'),
 Card(rank='5', suit='clubs'),
 Card(rank='6', 

![image.png](image/13-3.png)

MutableSequence ABC và superclass của nó từ collections.abc
- In nghiêng là abstract class/method

--> Để dùng ABCs, cần biết cái gì availble. Xem phần tiếp

### ABCs in the Standard Library

Đây là list các ABCs:
https://docs.python.org/3/library/collections.abc.html#collections-abstract-base-classes

- Hầu hết ở collection.abc module

Figure 13-4: 17 ABCs defined trong `collections.abc`

![](image/13-4.png)

- Iterable, Container, Sized:
    - Mọi Collection đề kế thừa từ ABCs này và imple các protocol thích hợp
    - Iterable: `__iter__`
    - Container: `in` - `__contains__`
    - Sized: `len()` - `__len__`

- Sequence, Mapping, Set:
    - main immutable collection types, mỗi loại có 1 mutable subclass

- MappingView
    - Các object trả về từ `.items()`, `.keys()`, và `.values()` define trong `ItemsView`, `KeysView` và `ValuesView`

- Callable, Hashable
    - Hỗ trợ type check object là callable/hashable.
        - `callable(obj)` hơn là `insinstance(obj, Callable)`
        - `hash(obj)` hơn là `insinstance(obj, Hashable)`:
                - False -> object ko thể hash
                - True -> obj có imple `__hash__` --> có TH dương tính giả (false positive)
                - bởi vậy, dùng `hash(obj)`


Ok, giờ thực hành imple goose typing.
Mục tiêu để hiểu cách đọc source code của ABCs

### Defining and Using an ABC

> "ABC, giống như bộ mô tả và siêu dữ liệu, là công cụ để xây dựng khung. Do đó, chỉ một số ít các nhà phát triển Python có thể tạo ABC mà không áp đặt các giới hạn vô lý và làm việc không cần thiết cho các lập trình viên đồng nghiệp."

Gọi ABCs thay vì các kiểu rõ ràng trong func argument type hint mang lại sự linh hoạt hơn cho người gọi.

VD: tạo ADAM - ads management framework. Hỗ trợ nonrepeating random-picking
- ABCs trên là `Tombola` (~ xổ số)
    - 2 abstract methods: (in nghiêng)
        - `.load(...)`
        - `.pick(...)`
    - 2 concrete methods:
        - `.loaded(...)`
        - `.inspect()`: Return `tuple` từ các items hiện có trong container

Hình 13-5 Tombola ABC và ba triển khai cụ thể.

![](image/13-5.png)

- Nét đứt --> dùng cho interface implementation.


In [2]:
# tag::TOMBOLA_ABC[]

import abc

class Tombola(abc.ABC):  # <1> define an ABC, subclass abc.ABC

    @abc.abstractmethod
    def load(self, iterable):  # <2> Abstract method đc mask bằng @abc.abstractmethod
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self):  # <3>
        """Remove item at random, returning it.

        This method should raise `LookupError` when the instance is empty.
        """

    def loaded(self):  # <4>
        """Return `True` if there's at least 1 item, `False` otherwise."""
        # Chưa tối ưu, build 1 tuple chỉ để check bool
        return bool(self.inspect())  # <5>

    def inspect(self):
        # Chưa tối ưu, mục tiêu chỉ là chỉ ra
        # các method cụ thể chỉ phụ thuộc vào các method trong interface
        # Các subclass có thể ghi đè
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:  # <6>
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)  # <7>
        return tuple(items)


# end::TOMBOLA_ABC[]

# 5: Concrete methods in an ABC must rely only on the interface defined by the ABC
# (i.e., other concrete or abstract methods or properties of the ABC).

TIP
1 abstract method có thể có implementation. Subclass vẫn phải viết imple của riêng nó, nhưng vẫn có thể invoke abstract method bằng `super()` thay vì triển khai từ đầu.

![](image/13-6.png)

In [3]:
class Fake(Tombola):
    def pick(self):
        return 13

f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract method load

### ABC Syntax Details

For example, the preferred way to declare an abstract class method is:
```python
class MyABC(abc.ABC):
    @classmethod
    @abc.abstractmethod
    def an_abstract_classmethod(cls, ...):
        pass
```

### Subclassing an ABC
Triển khai 2 subclass cụ thể

In [ ]:
# tag::TOMBOLA_BINGO[]

import random

class BingoCage(Tombola):  # <1>

    def __init__(self, items):
        self._randomizer = random.SystemRandom()  # <2>
        self._items = []
        self.load(items)  # <3>

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)  # <4> Thay vì dùng ramdom.shuffle() thuần, ta dùng `.shuffle()` của SystemRandom

    def pick(self):  # <5>
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):  # <6> Thêm vào cho vui, để shortcut to bingo.pick(): bingo()
        self.pick()

# end::TOMBOLA_BINGO[]
# Vẫn dùng 2 method cụ thể mà chưa tối ưu từ Tombola

Ví dụ 13-10 cho thấy một cách triển khai giao diện Tombola rất khác nhưng không kém phần hợp lệ. Thay vì xáo trộn các “quả bóng” và bật quả cuối cùng, LottoBlower bật từ một vị trí ngẫu nhiên.

In [4]:
# tag::LOTTERY_BLOWER[]

import random

class LottoBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)  # <1> Đc để cập trong lập trình phòng thủ, đồng thời tạo 1 bản sao, tránh iterable của client bị ảnh hưởng

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))  # <2>
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)  # <3>

    def loaded(self):  # <4>
        return bool(self._balls)

    def inspect(self):  # <5>
        return tuple(self._balls)


# end::LOTTERY_BLOWER[]

# 2: The random.randrange(…) function raises ValueError if the range is empty, so we catch that and throw LookupError instead, to be compatible with Tombola.

Phần này đi vào tính năng động quan trọng của gõ ngỗng:
- khai báo virtual subclasses bằng phương register method.

### A Virtual Subclass of an ABC

Khả năng đăng ký một lớp dưới dạng một lớp con ảo của một ABC, ngay cả khi nó không kế thừa từ nó.
- Class sẽ imple the interface được định nghĩa trong ABC

Figure 13-7. UML class diagram for the `TomboList`,
- a real subclass of `list`
- a virtual subclass of `Tombola`.

![](image/13-7.png)

In [5]:
from random import randrange

@Tombola.register  # <1> as a virtual subclass of Tombola
class TomboList(list):  # <2> extends list

    def pick(self):
        if self:  # <3> Kế thừ từ list, return True nếu list ko empty
            position = randrange(len(self))
            return self.pop(position)  # <4>
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend  # <5> Tombolist.load is the same as list.extend

    def loaded(self):
        return bool(self)  # <6>

    def inspect(self):
        return tuple(self)

# Tombola.register(TomboList)  # <7>
# Thay vì gọi bằng deco như ở trên, có thể gọi như cách ở <#7>,
# hữu ích khi register 1 class mà ko cần maintain , mà vẫn fulfill interface

In [6]:
issubclass(TomboList, Tombola)

True

In [7]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

Kế thừa được chỉ định bằng `__mro__` (Method Resolution Order). Nó list class_ superclass

In [8]:
TomboList.__mro__
# Tombola không có trong Tombolist.__mro__,
# -> vì vậy Tombolist không kế thừa bất kỳ phương thức nào từ Tombola.

(__main__.TomboList, list, object)

### Usage of register in Practice

Subclassing an ABC or registering with an ABC are both explicit ways of making our classes pass `issubclass` checks—as well as `isinstance` checks.

### Structural Typing with ABCs
Structural Typing:
- Nhìn vào structure của 1 object’s public interface để xác định kiểu.
- Dynamic/static duck typing are two approaches to structural typing.

ABC chủ yếu được sử dụng với nominal typing.
- `Sub` class inherits/register với `AnABC`, `AnABC` sẽ link với `Sub` class, vì thế ở runtime, `issubclass(AnABC, Sub)` returns `True`


In [9]:
# Cách mà Python biết subclass của ABC mà ko cần registration

class Struggle:
    def __len__(self): return 23

from collections import abc
isinstance(Struggle(), abc.Sized), issubclass(Struggle, abc.Sized)

(True, True)

Vì `abc.Sized` có `__subclasshook__` method , nó check xem `Sized` có imple `__len__` hay ko, nếu có, nó được mark là 1 virtual subclass of `abc.Sized`


```python

class Sized(metaclass=ABCMeta):

    __slots__ = ()

    @abstractmethod
    def __len__(self):
        return 0

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Sized:
            if any("__len__" in B.__dict__ for B in C.__mro__):
                return True
        return NotImplemented
```

Tuy nhiên, việc tự triển khai `__subclasshook__` là rất rủi ro.

Note
- https://peps.python.org/pep-0008/#function-and-method-arguments
- Always use `self` for the first argument to instance methods.
- Always use `cls` for the first argument to class methods.

## Static Protocols

`typing.Protocol` subclasses — useful for static and runtime type checking.

## The Typed double Function


In [11]:
def double(x):
    return x * 2

print(double(2))

print(double('A'))

from fractions import Fraction
print(double(Fraction(2, 5)))

4
AA
4/5


Nhờ duck typing, `double` còn hoạt động được cả với type mới

```
>>> from vector_v7 import Vector
>>> double(Vector([11.0, 12.0, 13.0]))
Vector([22.0, 24.0, 26.0])
```

Duck typing could not be described by type hints before Python 3.8.

Với `typing.Protocol` we can tell `Mypy` that `double` takes an argument `x` that supports `x * 2`

In [13]:
from typing import TypeVar, Protocol

T = TypeVar('T') # 1

class Repeatable(Protocol):
    def __mul__(self: T, repeat_count: int) -> T: ... # 2 Kiểu trả về giống với kiểu nhận vào

RT = TypeVar('RT', bound=Repeatable) # 3

def double(x: RT) -> RT: # 4 Kiểu trả về giống kiểu nhận và và kiểu đó có khả năng nhân với 1 số
    return x * 2

# 1. use this T in the __mul__ signature.

# 2. `__mul__` is the essence of the `Repeatable` protocol.
# The `self` parameter is usually not annotated—its type is assumed to be the class.
# Here we use T to make sure the result type is the same as the type of `self`.
# Also, note that `repeat_count` is limited to int in this protocol.

# 3. RT type được bound vào `Repeatable`. Khi đó, type check sẽ yêu cầu RT imple `Repeatable`

# 4. Khi này, type check có thể verify `x` là 1 object có thể nhân đc với 1 số, và trả về giá trị cùng kiểu vs x.

### Runtime Checkable Static Protocols

`typing.Protocol` appears in the static checking area. Tuy nhiên khi define 1 `typing.Protocol` subclass, có thể dùng `@runtime_checkable` để dùng được `isinstance/issubclass` ở runtime.
- Vì `typing.Protocol` là 1 ABC + nó có `__subclasshook__`

Đây là 2/7 protocols trong `typing` module có thể runtime check.
- class typing.SupportsComplex
    - An ABC with one abstract method, `__complex__`.

- class typing.SupportsFloat
    - An ABC with one abstract method, `__float__`.

In [ ]:
# Example 13-14. typing.SupportsComplex protocol source code

@runtime_checkable
class SupportsComplex(Protocol):
    """An ABC with one abstract method __complex__."""
    __slots__ = ()

    @abstractmethod
    def __complex__(self) -> complex:
        pass

In [15]:
# Example 13-15. Using SupportsComplex at runtime
# complex: tạo 1 số phức từ phần thực / phần ảo

from typing import SupportsComplex
import numpy as np

c64 = np.complex64(3+4j)
isinstance(c64, complex) # None of the NumPy complex types subclass the built-in `complex`.

False

In [16]:
# But NumPy’s complex types implement `__complex__`,
# so they comply with the `SupportsComplex` protocol.
isinstance(c64, SupportsComplex)

True

In [18]:
# Therefore, you can create built-in `complex` objects from them.
c = complex(c64)

# However, the `complex` built-in type does not implement `__complex__`, although complex(c) works fine if c is a complex.
isinstance(c, SupportsComplex)

False

In [19]:
# Test c là complex hoặc SupportsComplex
isinstance(c, (complex, SupportsComplex))

True

## Limitations of Runtime Protocol Checks

Type hint bị bỏ qua ở runtime -> `isinstance`/`issubclass` bị ảnh hưởng đối với static protocols.
- Ví dụ: Bất kì class với `__float__` sẽ được coi là (ở runtime) là 1 virtual subclass của `SupportsFloat`, mặc dù `__float__` ko return `float`.

In [20]:
import sys
sys.version

'3.10.9 (main, Jan 11 2023, 09:18:20) [Clang 14.0.6 ]'

In [21]:
c = 3+4j
c.__float__

AttributeError: 'complex' object has no attribute '__float__'

In [22]:
c.__float__()

AttributeError: 'complex' object has no attribute '__float__'

### Designing a Static Protocol

In [2]:
# Example 13-18. randompick.py: definition of RandomPicker

from typing import Protocol, runtime_checkable, Any

@runtime_checkable
class RandomPicker(Protocol):
    def pick(self) -> Any: ...


In [30]:
# Example 13-19. randompick_test.py: RandomPicker in use

import random
from typing import Any, Iterable, TYPE_CHECKING

#from randompick import RandomPicker # <1>

class SimplePicker:  # <2>
    def __init__(self, items: Iterable) -> None:
        self._items = list(items)
        random.shuffle(self._items)

    def pick(self) -> Any:  # <3>
        return self._items.pop()

def test_isinstance() -> None:  # <4>
    popper: RandomPicker = SimplePicker([1])  # <5>
    print(isinstance(popper, RandomPicker))
    assert isinstance(popper, RandomPicker)  # <6>

def test_item_type() -> None:  # <7>
    items = [1, 2]
    popper = SimplePicker(items)
    item = popper.pick()
    assert item in items
    if TYPE_CHECKING:
        reveal_type(item)  # <8>
    print(isinstance(item, int))
    assert isinstance(item, int)

# 1: Ko cần thiết phải import static protocal để định nghĩa 1 class mà impl nó. Import để đây chỉ là để cho `test_isinstance`

# 2: SimplePicker implements RandomPicker—but it does not subclass it.
# This is static duck typing in action.

# 3: Any is the default return type, it does make it more clear that we are implementing the RandomPicker protocol

# 4: Don’t forget to add -> None hints to your tests if you want Mypy to look at them.

# 8: reveal_type is a “magic” function recognized by Mypy. để bổ sung note

In [31]:
test_isinstance()

True


In [32]:
test_item_type()

True


Mypy also shows the result of the `reveal_type` on the item returned by `pick`:
```
~/Doc/G/example-code-2e/13/typing master +4 !1 ❯ mypy randompick_test.py     Py python310 23:30:36
randompick_test.py:24: note: Revealed type is "Any"
Success: no issues found in 1 source file
```

### Best Practices for Protocol Design

"Narrow protocols" are more useful—often such protocols have a single method, rarely more than a couple of methods. .

"Client code protocols":
- a protocol is defined near the function that uses it, instead of being defined in a library.
- easy to create new types to call that function
- good for extensibility and testing

Cả 2 đều tránh sự kết hợp chặt chẽ không cần thiết, để phù hợp với quy tắc:
- “Client không nên bị buộc phải phụ thuộc vào các giao diện mà họ không sử dụng.”

Naming convention cho static protocals:
- Tên đơn giản (e.g., Iterator, Container).
- `Supports+X` cho protocal cung cấp callable methods (e.g., `SupportsInt`, `SupportsRead`, `SupportsReadSeek`)
- `Has+X` cho protocals có các attr có thể đọc/ghi hoặc get/set method (e.g., `HasItems`, `HasFileno`).
- Nếu protocal có 1 method, có method name là Verd --> thêm "er"/"or" để thành Noun
    - instead of `SupportsRead`, have `Reader`


### Extending a Protocol

Thay vì thêm các phương thức vào giao thức ban đầu, tốt hơn là lấy một giao thức mới từ giao thức đó

In [3]:
# Example 13-20. randompickload.py: extending RandomPicker

from typing import Protocol, runtime_checkable

@runtime_checkable
class LoadableRandomPicker(RandomPicker, Protocol): # 2
    def load(self, Iterable) -> None: ...

# 2: Mỗi giao thức phải đặt tên rõ ràng typing.Protocol là một trong các lớp cơ sở của nó ngoài giao thức mà chúng tôi đang mở rộng.
# Điều này khác với cách hoạt động của tính kế thừa trong Python

### The numbers ABCs and Numeric Protocols

- The `numbers` ABCs are fine for runtime type checking, but unsuitable for static typing.
- The numeric static protocols `SupportsComplex`, `SupportsFloat`, etc. work well for static typing, but are unreliable for runtime type checking when complex numbers are involved.

# Chapter Summary

Từ 4 kiểu typing ở hình 13-1, ta so sánh dynamic protocols (duck typing) and static protocols (static duck typing):
- Điểm chung:
    - 1 class ko bắt buộc phải khai báo rõ ràng việc support cho 1 protocal cụ thể
        - chỉ cần đơn giản implement các methods cần thiết

Phần "Programming Duck" ta thấy:
- Cách tạo class để impl 1 protocal ở runtime thông qua monkey patching
- Các gợi ý cho việc lập trình phòng thủ
    - Gồm xác định structure type bằng `try/except` and failing fast, mà ko cần check `isinstance`/`hasattr`

Giới thiệu Goose typing
- Cách subclass existing ABCs
- Các ABCs trong thư viện chuẩn
- Tạo ABC riêng
    - Qua subclassing và qua registration
- `__subclasshook__` cho phép ABCs hỗ trợ structure typing

Phần "Static Protocals"
- Tiếp tục nói về `static duck typing`
- `@runtime_checkable` decorator tận dụng `__subclasshook__` để support structural typing at runtime
- Cách design và code static protocal
- Kết thúc với Numeric Protocals
    - Về tình trạng vô chủ của tháp số và một số thiếu sót hiện có của giải pháp thay thế được đề xuất

Thông điệp của chap này là
- Có 4 cách để lập trình interfaces trong Python và ta cần tìm ra cái phù hợp vs chương trình của mình